___
# Ciência dos Dados - PROJETO 1

## Por: Matteo Iannoni
___

## Titulo: Comparando como é efetuado o uso de internet em domicilios mais e menos saturados de eletronicos

___
## Introdução

Descreva aqui o que pretende analisar!

Capriche no português!! 
Se quiser pode utilizar pesquisas semelhantes publicadas em sites confiáveis!!


___
## Análise e Resultados

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Matteo Iannoni\Documents\GitHub\SDD_P1\Projeto 1


In [27]:
#Se usar o arquivo descompactado
DataFrame = pd.read_csv('DOM2015/DOM2015.csv',sep=',')
# df_full = DataFrame.iloc[0:3,0:5]#fileiras e colunas especificas: [1,2,3,4],[0,1,2,3]]
# DataFrame["V0201"]
# df_filtro_grosso = pd.DataFrame({"Especie de domicílio":DataFrame["V0201"],
#                                 "Numero de comodos no domicílio":DataFrame["V0205"],
#                                 "Numero de comodos servindo de dormitorio":DataFrame["V0206"],
#                                 "Condicao de ocupacao do domicilio":DataFrame["V0207"],
#                                 "Aluguel mensal pago no mes de referencia":DataFrame["V0208"],
#                                 "Prestacao mensal paga no mes de referencia":DataFrame["V0209"],
#                                 "Forma de iluminacao do domicilio":DataFrame["V0219"],
#                                 "Tem telefone movel celular":DataFrame["V0220"],
#                                 "Tem radio":DataFrame["V0225"],
#                                 "Tem televisao em cores":DataFrame["V0226"]
#                                 "numero de televisoes de tela fina":DataFrame["V02270"],
#                                 "Tem aparelho de DVD":DataFrame["V2027"],
#                                 "Tem microcomputador":DataFrame["V0231"],
#                                 "Microcomputador é uzado pra acessar a internet":DataFrame["V0232"],
#                                 "Tem tablet":DataFrame["V02321"],
#                                 "Tem acesso à internet no proprio domicilio":DataFrame["V02322"],
#                                 "Tem carro ou motocicleta de uso pessoal":DataFrame["V2032"],
#                                 "Rendimento mensal domiciliar":DataFrame["V4614"],
#                                 "Rendimento mensal domiciliar per capita":DataFrame["V4621"],
#                                 "Faixa do rendimento domiciliar per capita":DataFrame["V4622"],
#                                 "Para acessar à Internet utiliza-se microcomputador":DataFrame["V02323"],
#                                 "Para acessar à Internet utiliza-se telefone celular":DataFrame["V02324"],
#                                 "Para acessar à Internet utiliza-se tablet":DataFrame["V02325"],
#                                 "Para acessar à Internet utiliza-se tv":DataFrame["V02326"],
#                                 "Para acessar à Internet utiliza-se outro equipamento eletrônico":DataFrame["V02327"],
#                                 "O acesso à Internet é feito através de sinal de rede celular 3G ou 4G":DataFrame["V02424"],
#                                 "O acesso à Internet é feito através de conexão discada por linha telefônica":DataFrame["V02425"],
#                                 "O acesso à Internet é feito por banda larga":DataFrame["V02426"]})
df_filtro_grosso = pd.DataFrame({"Domicílio":DataFrame["V0201"],
                                "Forma de iluminacao do domicilio":DataFrame["V0219"],
                                "Tem telefone movel celular":DataFrame["V0220"],
                                "Tem radio":DataFrame["V0225"],
                                "Tem televisao em cores":DataFrame["V0226"],
                                "numero de televisoes de tela fina":DataFrame["V02270"],
                                "Tem aparelho de DVD":DataFrame["V2027"],
                                "Tem microcomputador":DataFrame["V0231"],
                                "Microcomputador é uzado pra acessar a internet":DataFrame["V0232"],
                                "Tem tablet":DataFrame["V02321"],
                                "Tem acesso à internet no proprio domicilio":DataFrame["V02322"],
                                "Para acessar à Internet utiliza-se microcomputador":DataFrame["V02323"],
                                "Para acessar à Internet utiliza-se telefone celular":DataFrame["V02324"],
                                "Para acessar à Internet utiliza-se tablet":DataFrame["V02325"],
                                "Para acessar à Internet utiliza-se tv":DataFrame["V02326"],
                                "Para acessar à Internet utiliza-se outro equipamento eletrônico":DataFrame["V02327"],
                                "O acesso à Internet é feito através de sinal de rede celular 3G ou 4G":DataFrame["V02424"],
                                "O acesso à Internet é feito através de conexão discada por linha telefônica":DataFrame["V02425"],
                                "O acesso à Internet é feito por banda larga":DataFrame["V02426"]})

df_filtro_grosso = df_filtro_grosso[df_filtro_grosso.Domicílio == 1]
df_filtro_grosso.to_excel('filtro_grosso.xlsx', sheet_name='Sheet1')

# df_filtro_grosso.to_excel('filtro_grosso.xlsx', sheet_name='Sheet1')
# df_filtro_grosso
#Se usar o arquivo compactado como está no dropbox púbico
#pd.read_csv('DOM2015csv.zip', compression='zip', header=0, sep=',', quotechar='"')

___
## Conclusão 

Descreva aqui suas conclusões!